In [1]:
cd ..

/gpfs/home/gdb20/code/batch-manager


In [2]:
import helpers
import input_files
import input_combi

In [3]:
global_configs = { '': {
        'xyz_directory' : '/gpfs/home/gdb20/code/mol-maker/data/lib/frag',
        'charge': 0,
        'solvent' : 'THF',
    },
}
#need:
#to be able to group these into directories by name; 
#make name part of directory path and not basename as a flag

#need for now: make one of these from a directory of xyz files

molecules = {'benzene':{
                'xyz_file' : 'benzene.xyz',
                },
             'p-quinodimethane':{
                'xyz_file' : 'p_quinodimethane.xyz',
                },
             'thiophene': {
                 'xyz_file' : 'thiophene.xyz',
                },
            '!directories' : True,
            }

#################### PROGRAM SETTINGS ##############################

crest_settings = {'crest' : {
                    'program' : 'CREST',
                    'functional' : 'gfnff',
                    'quick' : 'mquick',
                    'reopt' : True,
                    'cluster' : True,
                    }
                 }


ORCA_settings = {
            'orca' : {
                'program' : 'orca',
                'integration_grid' : 'DEFGRID3', #tortoise and hare
                'num_cores' : 12,
                '!coords_from': 'crest', 
                '!xyz_file': 'crest_best.xyz', #
                },                      #where the whole name up to and including this phrase is included
           }


################# SCF and spin settings ######################

spin_scf_settings = {
            'triplet' : {
                'spin_multiplicity' : 3,
                'uks' : True,
            },
            'singlet' : {
                'spin_multiplicity' : 3,
                'broken_symmetry' : True,
                'uks' : True,
            }
}


################# LEVEL OF THEORY SETTINGS ##########################

#FOR ORCA CONFIG:
#ADD SCF CONVERGENCE TOLERANCE
#ADD PRINTING VERBOSITY
#ADD MOINP OPTION


#NEED: orbitals_from option

#these could just have a basename that is only the stuff not already included in their group directories
#less redundant, maybe easier to manipulate
#but most of all, this is how every other CC program and driver I've ever seen does it

methods = {    
    # 'r2SCAN-3c_opt_freq' : {
    #             'functional' : 'r2scan-3c',
    #             'basis_set' : None,
    #             },
    'wB97X-3c_sp' : { #this one will include options for rendering orbitals,
                #and densities
                'functional' : 'wB97X-3c',
                'basis_set' : None,
                'natural_orbitals' : True,
                'verbosity' : 'NormalPrint',
                'blocks' : {'plots': [
                                'dim1 100',
                                'dim2 100',
                                'dim3 100',
                                'Format Gaussian_Cube',
                                'SpinDens("spindensity");',
                                'ElDens("electrondensity");',
                                ],
                            'output': [
                                'print[P_UNO_OccNum] 1'
                                ],
                            },
                '!coords_from' : 'orca_r2SCAN-3c_opt_freq',
                '!xyz_file' : 'orca_r2SCAN-3c_opt_freq.xyz',
                '!orbitals_from' : 'orca_r2SCAN-3c_opt_freq',
                },
    'wB97X-3c_tddft' : {
               'functional' : 'wB97X-3c',
               'basis_set' : None,
               '!coords_from' : 'orca_r2SCAN-3c_opt_freq',
               '!xyz_file' : 'orca_r2SCAN-3c_opt_freq.xyz',
               '!orbitals_from' : 'orca_r2SCAN-3c_opt_freq',
               },
}

root = '/gpfs/home/gdb20/computations/mike/stgaps_oct'
configs, flags = input_combi.sort_flags([global_configs,molecules,ORCA_settings,methods])
inputs = input_combi.iterate_inputs(configs,flags)
crestconfigs,flags = input_combi.sort_flags([global_configs,molecules,crest_settings])
crinputs = input_combi.iterate_inputs(crestconfigs,flags)
#some script for requesting cube files

################### SOLVENT SETTINGS #############################

solvents = { #TODO: implement solvent model syntax in ORCA
    #            'gas' : {
    #             'solvent' : None
    #             },
            'thf' : {
                'solvent' : 'thf'
                },
           }

In [4]:


# input_combi.generate_batchfile(inputs, root, 'batchfile.csv')
# input_combi.generate_batchfile(crinputs, root, 'batchfile.csv')
input_combi.write_input_array(inputs,root)
#input_combi.write_input_array(crinputs,root)

In [7]:
inputs

[{'xyz_directory': '/gpfs/home/gdb20/code/mol-maker/data/lib/frag',
  'charge': 0,
  'solvent': 'THF',
  'write_directory': '/gpfs/home/gdb20/computations/mike/stgaps_oct/benzene/orca_r2SCAN-3c_opt_freq',
  'xyz_file': 'benzene.xyz',
  'program': 'orca',
  'integration_grid': 'DEFGRID3',
  'other_keywords': ['VeryTightSCF',
   'Normalprint',
   'noiter',
   'MOREAD',
   'Normalprint',
   'noiter',
   'MOREAD',
   'Normalprint',
   'noiter',
   'MOREAD'],
  '!coords_from': 'crest',
  '!xyz_file': 'crest_best.xyz',
  'functional': 'r2scan-3c',
  'basis_set': None,
  'job_basename': 'orca_r2SCAN-3c_opt_freq'},
 {'xyz_directory': '/gpfs/home/gdb20/code/mol-maker/data/lib/frag',
  'charge': 0,
  'solvent': 'THF',
  'write_directory': '/gpfs/home/gdb20/computations/mike/stgaps_oct/benzene/orca_wB97X-3c_sp',
  'xyz_file': 'benzene.xyz',
  'program': 'orca',
  'integration_grid': 'DEFGRID3',
  'other_keywords': ['VeryTightSCF',
   'Normalprint',
   'noiter',
   'MOREAD',
   'Normalprint',
   '